In [1]:
#Import the required libraries
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract

#Natural Language Processing libraries
from spacy.matcher import PhraseMatcher
import nltk
import re as re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
# Importing tesseract from the location
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe" 

In [12]:
#Testing on one image
img = cv2.imread('./Dataset/Test213.jpg')
#gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#img = cv2.medianBlur(img,1)
sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
sharpen = cv2.filter2D(img, -1, sharpen_kernel)
text = pytesseract.image_to_string(sharpen, lang='eng')
print(text)
#cv2.imshow("img",img)
cv2.imshow("sharpen",sharpen)
cv2.waitKey(0)
cv2.destroyWindow('sharpen')


a


In [4]:
#Performing OCR on base image
import cv2
import os
import glob
img_dir = 'C:/Users/kanum/Desktop/Data_Science_Everyday/Hackathons/Love is Love/Dataset' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
filename = []
for f1 in files:
    file = os.path.basename(f1)
    filename.append(file)
    img = cv2.imread(f1)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv2.filter2D(img, -1, sharpen_kernel)
    text = pytesseract.image_to_string(sharpen, lang='eng')
    data.append(text)

In [5]:
inp = pd.DataFrame()
inp['Filename'] = filename
inp['text'] = data
print(inp['text'].value_counts())
inp.head(5)


                                                                                                                                                                                                                                                                                                                                                                  96
Being gay is not a crime\n\nand itis not a sin. Stop using\nGod to justify your prejudice.\nReligion is about loving one\nie Ola cey (Usa lee Cire\nfor an excuse to hate.\n\ncl Being Gay S Proud Quotes\nwww. geckoandfly.com\n\nint as 4                                                                                                                        2
Being gay is like being\n\nleft handed. Some people\nare, most people aren't\nand nobody really knows\nwhy. It’s not right or wrong,\neS Umea enses ey\n\n2 Being Gay & Proud Quotes\nwww.geckoandfly. com                                                                                    

,Filename,text
0,Test100.jpg,
1,Test1001.jpg,"When people ask\nwhat I see in you,\nI just sm..."
2,Test1012.jpg,
3,Test1022.jpg,LOVE\n©
4,Test103.jpg,Aj\n: ALO\naly\n\n~ 160 YEARS OF ATLANTIG STORIES


I have extracted text from the images, I have performed OCR on images(I didn't preprocess the images yet). Now I can start performing Natural Language Processing on the extracted text.For extracting sentiment from the text I am implementing VADAR (Valence Aware Dictionary and sEntiment Reasoner)

VADER ( Valence Aware Dictionary for Sentiment Reasoning) is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.
VADER sentimental analysis relies on a dictionary that maps lexical features to emotion intensities known as sentiment scores. The sentiment score of a text can be obtained by summing up the intensity of each word in the text.
For example- Words like ‘love’, ‘enjoy’, ‘happy’, ‘like’ all convey a positive sentiment. Also VADER is intelligent enough to understand the basic context of these words, such as “did not love” as a negative statement. It also understands the emphasis of capitalization and punctuation, such as “ENJOY”

__VADER provides a positive, negative, and neutral sentiment scalar, as well as a compound sentiment vector that combines the three__

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

VADER’s SentimentIntensityAnalyzer() takes in a string and returns a dictionary of scores in each of four categories:
- negative
- neutral
- positive
- compound (computed by normalizing the scores above)

In [7]:
analyzer = SentimentIntensityAnalyzer()

In [8]:
#Applying on a simple text
a = inp['text'][1]
analyzer.polarity_scores(a)

{'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'compound': 0.7717}

In [9]:
#Adding scores dictionary to the input data
inp['scores'] = inp['text'].apply(lambda text: analyzer.polarity_scores(text))
inp.head()

,Filename,text,scores
0,Test100.jpg,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
1,Test1001.jpg,"When people ask\nwhat I see in you,\nI just sm...","{'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'comp..."
2,Test1012.jpg,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
3,Test1022.jpg,LOVE\n©,"{'neg': 0.0, 'neu': 0.169, 'pos': 0.831, 'comp..."
4,Test103.jpg,Aj\n: ALO\naly\n\n~ 160 YEARS OF ATLANTIG STORIES,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [10]:
#Cleaned text
def clean_text(text):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', text).split())

inp["text"] = inp['text'].apply(lambda x: clean_text(x))

def sentiment_scores(text): 
  
    # Create a SentimentIntensityAnalyzer object. 
    analyzer_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = analyzer_obj.polarity_scores(text) 
      
    if sentiment_dict['compound'] > 0:
        return 'Positive'
    elif sentiment_dict['compound'] < 0:
        return 'Negative'
    else:
        return 'Random'

inp["Category"] = inp["text"].apply(lambda x: sentiment_scores(x))

print(inp.head(5))
  

       Filename                                               text  \
0   Test100.jpg                                                      
1  Test1001.jpg  When people ask what I see in you I just smile...   
2  Test1012.jpg                                                      
3  Test1022.jpg                                               LOVE   
4   Test103.jpg           Aj ALO aly 160 YEARS OF ATLANTIG STORIES   

                                              scores  Category  
0  {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...    Random  
1  {'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'comp...  Positive  
2  {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...    Random  
3  {'neg': 0.0, 'neu': 0.169, 'pos': 0.831, 'comp...  Positive  
4  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...    Random  


In [11]:
inp.to_csv("inp_sharpen.csv",index = False)

In [12]:
submission = inp.drop(['text','scores'],axis = 1)
submission

,Filename,Category
0,Test100.jpg,Random
1,Test1001.jpg,Positive
2,Test1012.jpg,Random
3,Test1022.jpg,Positive
4,Test103.jpg,Random
...,...,...
234,Test945.jpg,Negative
235,Test946.jpg,Negative
236,Test957.jpg,Random
237,Test979.jpg,Random


In [13]:
submission.to_csv("submission_ct_sh.csv",index = False)